# ENTREGABLE 4

# INSTRUCCIONES

Utilizar el archivo CSV (`dataset_banco_clean.csv`) con 45189 filas y 17 columnas y aplicar las técnicas de normalización del entregable 3.

In [ ]:
# imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


In [ ]:
# Lectura
from google.colab import drive
import pandas as pd

drive.mount('/gdrive')

ruta = "/gdrive/My Drive/ejercicios_ml/clasificacion/dataset_banco_clean.csv"
data = pd.read_csv(ruta)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
print(data.shape)
data.head()

(45189, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5,may,261.0,1,-1.0,0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5,may,151.0,1,-1.0,0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5,may,76.0,1,-1.0,0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5,may,92.0,1,-1.0,0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5,may,198.0,1,-1.0,0,unknown,no


# Objetivo

Generar un model de clasificación capaz de predecir la clase de flor en función de las carácterísticas del dataset

* Aplicar las técnicas oportunas de procesamiento de datos

* Generar split de los datos

* Valorar diferentes modelos de clasificación

* Comparación entre modelos

* Ensemble

* Métricas

* Conclusiones finales

# Preprocesamiento de datos

1. Manejo de datos nulos
2. Codificación de variables categoricas
3. Normalización y estandarización
4. Manejo de outliers


In [ ]:
# Comprobar valores nulos
print(data.isnull().sum())

# Información general del dataset
data.info()


age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45189 entries, 0 to 45188
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        45189 non-null  int64  
 1   job        45189 non-null  object 
 2   marital    45189 non-null  object 
 3   education  45189 non-null  object 
 4   default    45189 non-null  object 
 5   balance    45189 non-null  float64
 6   housing    45189 non-null  object 
 7   loan       45189 non-null  object 
 8   contact    45189 non-null  object 
 9   day        45189 non-null  int64  
 10  month      45189 non-null  object 
 11  duration   45189 non-null  float64
 12  campaign   45189 non-null  int64  
 13  pdays      45189 n

Dado que el conjunto de datos no contiene valores nulos, podemos proceder directamente con otras técnicas de preprocesamiento

Verificar valores faltantes

In [ ]:
# Verificar valores faltantes por columna
missing_values = data.isnull().sum()
print(missing_values)


age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64


Aunque has verificado que no hay valores nulos en tu conjunto de datos, es posible que haya otros tipos de datos faltantes que no sean NaN, como cadenas vacías (''), valores marcados como "unknown" o cualquier otro tipo de marcador que no se interprete como NaN por pandas

In [ ]:
for col in data.columns:
    print(f'Columna: {col}')
    print(data[col].unique())
    print()


Columna: age
[58 44 33 47 35 28 42 43 41 29 57 51 45 60 56 32 25 40 39 52 46 36 49 59
 53 37 50 54 55 48 24 38 31 30 27 34 23 26 61 22 21 20 66 62 83 75 67 70
 65 68 64 69 72 71 19 76 85 63 90 82 73 74 78 80 94 79 77 86 95 81 18 89
 84 87 92 93 88]

Columna: job
['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'administrative' 'services' 'self-employed' 'unemployed'
 'housemaid' 'student']

Columna: marital
['married' 'single' 'divorced']

Columna: education
['tertiary' 'secondary' 'unknown' 'primary']

Columna: default
['no' 'yes']

Columna: balance
[2.1430e+03 2.9000e+01 2.0000e+00 ... 8.2050e+03 1.4204e+04 1.6353e+04]

Columna: housing
['yes' 'no']

Columna: loan
['no' 'yes']

Columna: contact
['unknown' 'cellular' 'telephone']

Columna: day
[ 5  6  8  7 16  9 12 13 14 15 19 20 21 23 26 27 28 29 30  2  3  4 11 17
 18 24 25  1 10 22 31]

Columna: month
['may' 'nov' 'jun' 'jul' 'aug' 'oct' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']

Columna: duration
[ 261.  151.

No hay valores nulos (NaN) en ninguna de las columnas, pero hay valores marcados como "unknown" en las columnas job, education, contact y poutcome

Para poder tomar la mejor decisión de qué hacer con esos datos

In [ ]:
unknown_counts = {}
for col in data.columns:
    if 'unknown' in data[col].unique():
        unknown_counts[col] = data[col].value_counts()['unknown']

print("Cantidad de datos desconocidos por columna:")
for col, count in unknown_counts.items():
    print(f"{col}: {count}")


Cantidad de datos desconocidos por columna:
job: 288
education: 1857
contact: 13011
poutcome: 36943


In [ ]:
total_records = len(data)
print("Porcentaje de datos desconocidos por columna:")
for col, count in unknown_counts.items():
    percentage = (count / total_records) * 100
    print(f"{col}: {percentage:.2f}%")


Porcentaje de datos desconocidos por columna:
job: 0.64%
education: 4.11%
contact: 28.79%
poutcome: 81.75%


job y education: Tienen relativamente pocos datos desconocidos (menos del 5% en ambos casos), por lo que realizaremos imputación basada en moda: Puedes reemplazar los valores "unknown" por la moda (el valor más frecuente) de cada columna respectiva.

contact: Con casi el 29% de los datos desconocidos, la estrategia para manejar estos valores es unificarlos en telephone ya que el método es el mismo "por telefono" pero se denomina de manera diferente.

poutcome: Con más del 81% de los datos desconocidos, agruparemos los valores "unknown" de la columna poutcome dentro de la categoría "other" ya que son valores que no pertenecen ni a "failure" ni a "success"

In [ ]:
# Consolidar los valores de la columna 'contact'
data['contact'] = 'telephone'

# Verificar que ahora todos los valores son 'telephone'
print(data['contact'].unique())


['telephone']


In [ ]:
# Imputación en la columna 'job'
job_mode = data['job'].mode()[0]  # Calcula la moda de la columna 'job'
data['job'] = data['job'].replace('unknown', job_mode)

# Imputación en la columna 'education'
education_mode = data['education'].mode()[0]  # Calcula la moda de la columna 'education'
data['education'] = data['education'].replace('unknown', education_mode)

# Verificación de los cambios
print("Valores únicos después de la imputación:")
print("Columna 'job':", data['job'].unique())
print("Columna 'education':", data['education'].unique())


Valores únicos después de la imputación:
Columna 'job': ['management' 'technician' 'entrepreneur' 'blue-collar' 'retired'
 'administrative' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
Columna 'education': ['tertiary' 'secondary' 'primary']


In [ ]:
# Reemplazar 'unknown' por 'other' en la columna 'poutcome'
data['poutcome'] = data['poutcome'].replace('unknown', 'other')

# Verificación de los cambios
print("Valores únicos después del reemplazo en 'poutcome':")
print(data['poutcome'].unique())


Valores únicos después del reemplazo en 'poutcome':
['other' 'failure' 'success']


In [ ]:


# Aplicar One-Hot Encoding a las variables categóricas
data_encoded = pd.get_dummies(data, columns=[
    'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome'
], drop_first=True)

# Mostrar las primeras filas del dataset codificado
print(data_encoded.head())


   age  balance  day  duration  campaign  pdays  previous   y  \
0   58   2143.0    5     261.0         1   -1.0         0  no   
1   44     29.0    5     151.0         1   -1.0         0  no   
2   33      2.0    5      76.0         1   -1.0         0  no   
3   47   1506.0    5      92.0         1   -1.0         0  no   
4   33      1.0    5     198.0         1   -1.0         0  no   

   job_blue-collar  job_entrepreneur  ...  month_jan  month_jul  month_jun  \
0            False             False  ...      False      False      False   
1            False             False  ...      False      False      False   
2            False              True  ...      False      False      False   
3             True             False  ...      False      False      False   
4             True             False  ...      False      False      False   

   month_mar  month_may  month_nov  month_oct  month_sep  poutcome_other  \
0      False       True      False      False      False        

In [ ]:
# Verificar los valores únicos en las columnas originales antes de la codificación
print(data['job'].unique())
print(data['month'].unique())
print(data['poutcome'].unique())

# Asegurar que las categorías están correctamente asociadas con las nuevas columnas binarias
# Por ejemplo, si la columna 'job' tenía categorías 'blue-collar', 'entrepreneur', etc., deberían aparecer como columnas binarias correspondientes.

# Si hay problemas con la codificación, revisar y ajustar las categorías o cómo se están codificando.

# Mostrar las primeras filas del dataset codificado para verificar
print(data_encoded.head())


['management' 'technician' 'entrepreneur' 'blue-collar' 'retired'
 'administrative' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
['may' 'nov' 'jun' 'jul' 'aug' 'oct' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
['other' 'failure' 'success']
   age  balance  day  duration  campaign  pdays  previous   y  \
0   58   2143.0    5     261.0         1   -1.0         0  no   
1   44     29.0    5     151.0         1   -1.0         0  no   
2   33      2.0    5      76.0         1   -1.0         0  no   
3   47   1506.0    5      92.0         1   -1.0         0  no   
4   33      1.0    5     198.0         1   -1.0         0  no   

   job_blue-collar  job_entrepreneur  ...  month_jan  month_jul  month_jun  \
0            False             False  ...      False      False      False   
1            False             False  ...      False      False      False   
2            False              True  ...      False      False      False   
3             True             False  ...    

# Normalización





In [ ]:
# Normalización de características numéricas con Min-Max Scaling
print("\nNormalización de características numéricas:")
numeric_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
scaler = MinMaxScaler()
data_encoded[numeric_columns] = scaler.fit_transform(data_encoded[numeric_columns])

# Mostrar las primeras filas del dataset con las características normalizadas
print(data_encoded.head())


Normalización de características numéricas:
        age   balance       day  duration  campaign  pdays  previous   y  \
0  0.519481  0.018975  0.133333  0.052878       0.0    0.0       0.0  no   
1  0.337662  0.015028  0.133333  0.030506       0.0    0.0       0.0  no   
2  0.194805  0.014977  0.133333  0.015253       0.0    0.0       0.0  no   
3  0.376623  0.017785  0.133333  0.018507       0.0    0.0       0.0  no   
4  0.194805  0.014975  0.133333  0.040065       0.0    0.0       0.0  no   

   job_blue-collar  job_entrepreneur  ...  month_jan  month_jul  month_jun  \
0            False             False  ...      False      False      False   
1            False             False  ...      False      False      False   
2            False              True  ...      False      False      False   
3             True             False  ...      False      False      False   
4             True             False  ...      False      False      False   

   month_mar  month_may  mont

In [ ]:
# Manejo básico de outliers: transformación logarítmica de la columna 'balance'
print("\nManejo básico de outliers:")
data_encoded['balance_log'] = np.log(data_encoded['balance'] + 1)  # Sumamos 1 para evitar log(0)

# Mostrar las primeras filas del dataset con la transformación logarítmica aplicada
print(data_encoded.head())



Manejo básico de outliers:
        age   balance       day  duration  campaign  pdays  previous   y  \
0  0.519481  0.018975  0.133333  0.052878       0.0    0.0       0.0  no   
1  0.337662  0.015028  0.133333  0.030506       0.0    0.0       0.0  no   
2  0.194805  0.014977  0.133333  0.015253       0.0    0.0       0.0  no   
3  0.376623  0.017785  0.133333  0.018507       0.0    0.0       0.0  no   
4  0.194805  0.014975  0.133333  0.040065       0.0    0.0       0.0  no   

   job_blue-collar  job_entrepreneur  ...  month_jul  month_jun  month_mar  \
0            False             False  ...      False      False      False   
1            False             False  ...      False      False      False   
2            False              True  ...      False      False      False   
3             True             False  ...      False      False      False   
4             True             False  ...      False      False      False   

   month_may  month_nov  month_oct  month_sep 

In [ ]:
# Importar train_test_split desde sklearn
from sklearn.model_selection import train_test_split

# Definir X (variables independientes) y y (variable dependiente)
X = data_encoded.drop('y', axis=1)  # Todas las columnas menos 'y' son características
y = data_encoded['y']  # 'y' es nuestra variable objetivo




In [ ]:
# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:

# Mostrar las dimensiones de los conjuntos de entrenamiento y prueba
print("Dimensiones de los conjuntos de entrenamiento y prueba:")
print("Conjunto de entrenamiento:", X_train.shape, y_train.shape)
print("Conjunto de prueba:", X_test.shape, y_test.shape)
X = data_encoded.drop(columns=['y'])
y = data_encoded['y'].apply(lambda x: 1 if x == 'yes' else 0)  # Convertir 'yes'/'no' a 1/0 para la clasificación binaria

Dimensiones de los conjuntos de entrenamiento y prueba:
Conjunto de entrenamiento: (36151, 38) (36151,)
Conjunto de prueba: (9038, 38) (9038,)


In [ ]:
# Importar los modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Importar métricas para evaluación
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Inicializar los modelos
models = {
       'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

In [ ]:
# Importar los modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Importar métricas para evaluación
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Entrenar y evaluar modelos
modelos = {
    "Logistic Regression": LogisticRegression(max_iter=500, solver='lbfgs'),  # Aumentar max_iter
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier()
}

# Función para entrenar y evaluar cada modelo
def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return accuracy, confusion, report

# Entrenar y evaluar cada modelo
results = {}
for model_name, model in models.items():
    print(f"Entrenando {model_name}...")
    accuracy, confusion, report = train_and_evaluate(model, X_train, y_train, X_test, y_test)
    results[model_name] = {
        'accuracy': accuracy,
        'confusion_matrix': confusion,
        'classification_report': report
    }
    print(f"{model_name} completado.\n")

# Mostrar resultados
for model_name, result in results.items():
    print(f"Modelo: {model_name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print(f"Confusion Matrix:\n{result['confusion_matrix']}")
    print(f"Classification Report:\n{result['classification_report']}\n")

Entrenando Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression completado.

Entrenando Decision Tree...
Decision Tree completado.

Entrenando Random Forest...
Random Forest completado.

Entrenando Gradient Boosting...
Gradient Boosting completado.

Modelo: Logistic Regression
Accuracy: 0.8989
Confusion Matrix:
[[7810  168]
 [ 746  314]]
Classification Report:
              precision    recall  f1-score   support

          no       0.91      0.98      0.94      7978
         yes       0.65      0.30      0.41      1060

    accuracy                           0.90      9038
   macro avg       0.78      0.64      0.68      9038
weighted avg       0.88      0.90      0.88      9038


Modelo: Decision Tree
Accuracy: 0.8717
Confusion Matrix:
[[7380  598]
 [ 562  498]]
Classification Report:
              precision    recall  f1-score   support

          no       0.93      0.93      0.93      7978
         yes       0.45      0.47      0.46      1060

    accuracy                           0.87      9038
   macro avg       0.69     

Para mejorar la precisión del modelo, podemos aplicar métodos de ensemble, como por ejemplo Voting Classifier, que combina varios modelos para obtener predicciones más precisas.

In [ ]:
# Importar VotingClassifier
from sklearn.ensemble import VotingClassifier

# Inicializar los modelos para el VotingClassifier
estimators = [
    ('lr', LogisticRegression()),
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42))
]

# Inicializar el VotingClassifier
ensemble_model = VotingClassifier(estimators)

# Entrenar y evaluar el modelo de ensemble
print("Entrenando el Voting Classifier...")
accuracy, confusion, report = train_and_evaluate(ensemble_model, X_train, y_train, X_test, y_test)

# Mostrar resultados del ensemble
print(f"Voting Classifier - Accuracy: {accuracy:.4f}")
print(f"Confusion Matrix:\n{confusion}")
print(f"Classification Report:\n{report}")


Entrenando el Voting Classifier...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Voting Classifier - Accuracy: 0.9025
Confusion Matrix:
[[7834  144]
 [ 737  323]]
Classification Report:
              precision    recall  f1-score   support

          no       0.91      0.98      0.95      7978
         yes       0.69      0.30      0.42      1060

    accuracy                           0.90      9038
   macro avg       0.80      0.64      0.68      9038
weighted avg       0.89      0.90      0.89      9038



Conclusiones:

En el procesamiento de datos se ha llevado a cabo:

1. Manejo de 'unknown' y normalización: reemplazo de los valores 'unknown' en las columnas 'job', 'education', 'contact', y 'poutcome' con valores más representativos ('mode' para 'job' y 'education', 'other' para 'poutcome').

2. Codificación One-Hot para codificar las variables categóricas especificadas en categorical_cols.

3. Normalización Min-Max Scaling (MinMaxScaler) a las variables numéricas especificadas en numeric_cols.

4. Unión de datos: Combinación de las columnas codificadas y normalizadas con las columnas restantes del conjunto de datos original para formar data_encoded


En el entrenamiento del modelo:

1. Se dividió el conjunto de datos en conjuntos de entrenamiento (X_train, y_train) y prueba (X_test, y_test) con una proporción del 80% para entrenamiento y 20% para prueba. Esto asegura que los modelos se evalúen de manera adecuada en datos no vistos.

2. Se importaron varios modelos de clasificación (Logistic Regression, Decision Tree, Random Forest, Gradient Boosting) y se inicializaron dentro del diccionario models.
Cada modelo se entrenó utilizando X_train y y_train, luego se realizaron predicciones en X_test y se evaluaron utilizando métricas como precisión (accuracy), matriz de confusión (confusion_matrix) y reporte de clasificación (classification_report).

3. Se creó un Voting Classifier (ensemble_model) que combina las predicciones de varios modelos (Logistic Regression, Decision Tree, Random Forest, Gradient Boosting) utilizando votación mayoritaria.
El Voting Classifier también se entrenó con X_train y y_train, y se evaluó su desempeño en X_test, almacenando métricas de evaluación (accuracy_ensemble, confusion_ensemble, report_ensemble).

Los modelos individualmente y el Voting Classifier tienen un desempeño decente en términos de precisión (accuracy). El Random Forest y el Gradient Boosting obtienen una precisión ligeramente mejor que los otros modelos.

Para mejorar aún más el desempeño del modelo se podrías ajustar los hiperparámetros de los modelos, como la profundidad máxima del árbol en Decision Tree, el número de árboles en Random Forest, etc.
y considerar técnicas como el ajuste de pesos o la validación cruzada.